# Dependencies

In [2]:
import logging
import warnings
warnings.filterwarnings("ignore")
logging.getLogger('mne').setLevel(logging.WARNING)

import os
import mne
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib qt
# import torch
# import snntorch as snn
# from snntorch import spikegen, spikeplot
import pycochleagram.cochleagram as cgram

# Directories and filepaths

In [5]:
main_folder = "ThesisFolder"
extras_folder = "Extras"

data_folder_py = os.path.join(extras_folder, "GAMEEMO_PY")
data_folder_mat = os.path.join(extras_folder,"GAMEEMO_MAT")
data_folder_fif = os.path.join(main_folder,"GAMEEMO_FIF")
data_folder_epoch = os.path.join(extras_folder,"GAMEEMO_EPOCH")

files_py = os.listdir(data_folder_py)
files_mat = os.listdir(data_folder_mat)
files_fif = os.listdir(data_folder_fif)

# Spike Encoder Workspace

In [8]:
# def dynamic_range_compression(audio, sample_rate, threshold_dB, ratio, attack=1e-6, release=1e-6):
#     threshold_linear = 10 ** (threshold_dB / 20.0)
    
#     gain = 1.0
#     attack_coeff = np.exp(-1.0 / (sample_rate * attack))
#     release_coeff = np.exp(-1.0 / (sample_rate * release))
    
#     compressed_audio = np.zeros_like(audio)
    
#     for i in range(len(audio)):
#         input_level = np.abs(audio[i])
        
#         if input_level > threshold_linear:
#             target_gain = threshold_linear + (input_level - threshold_linear) / ratio
#         else:
#             target_gain = input_level
        
#         if target_gain < gain:
#             gain = attack_coeff * gain + (1 - attack_coeff) * target_gain
#         else:
#             gain = release_coeff * gain + (1 - release_coeff) * target_gain
        
#         compressed_audio[i] = audio[i] * gain
    
#     return compressed_audio

def dynamic_range_compression(audio, sample_rate, threshold, ratio, gain_multiplier=1, attack=0.1, release=0.1):
    threshold_linear = 10 ** (threshold / 20.0)
    
    attack_coeff = np.exp(-1.0 / (sample_rate * attack))
    release_coeff = np.exp(-1.0 / (sample_rate * release))
    
    gain = np.ones_like(audio)*gain_multiplier
    compressed_audio = np.zeros_like(audio)
    
    chunk_size = 1024
    for start in range(0, len(audio), chunk_size):
        end = min(start + chunk_size, len(audio))
        chunk = audio[start:end]
        
        input_level = np.abs(chunk)
        
        target_gain = np.where(input_level > threshold_linear,
                               threshold_linear + (input_level - threshold_linear) / ratio,
                               input_level)
        
        gain[start:end] = np.where(target_gain < gain[start:end],
                                   attack_coeff * gain[start:end] + (1 - attack_coeff) * target_gain,
                                   release_coeff * gain[start:end] + (1 - release_coeff) * target_gain)
        
        compressed_audio[start:end] = chunk * gain[start:end]
    
    return compressed_audio


In [10]:
rawArray = []
for file in tqdm(files_fif):
    filename = os.path.join(data_folder_fif, file)
    raw = mne.io.read_raw_fif(filename, preload=True)
    raw.filter(8, 42) 

    raw_data = raw.get_data() * 1e6
    raw_data = raw_data
    
    for idx, _ in enumerate(raw_data):
        #apply soft-limiter
        # alpha_multiplier = 20
        # alpha = alpha_multiplier / np.max(raw_data[idx])
        # threshold = 27.0
        # raw_data[idx] = raw_data[idx] / (1 + (alpha * np.maximum(0, np.abs(raw_data[idx]) - threshold)))
        # raw_data[idx] = (raw_data[idx]-np.min(raw_data[idx]))/(np.max(raw_data[idx])-np.min(raw_data[idx]))
        raw_data[idx] = (raw_data[idx]-0)/(np.max(raw_data[idx])-0)
        raw_data[idx] = dynamic_range_compression(raw_data[idx], 128, -5, 20, 1, 0.8, 0.8)
    
    rawArray.append(raw_data)

rawArray =np.array(rawArray, dtype=np.float32)

  0%|                                                                                          | 0/112 [00:00<?, ?it/s]

Opening raw data file ThesisFolder\GAMEEMO_FIF\S01G1AllChannels_raw.fif...
    Range : 0 ... 38251 =      0.000 ...   298.836 secs
Ready.
Reading 0 ... 38251  =      0.000 ...   298.836 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 42 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 42.00 Hz
- Upper transition bandwidth: 10.50 Hz (-6 dB cutoff frequency: 47.25 Hz)
- Filter length: 213 samples (1.664 s)



  1%|▋                                                                                 | 1/112 [00:00<00:33,  3.28it/s]

Opening raw data file ThesisFolder\GAMEEMO_FIF\S01G2AllChannels_raw.fif...
    Range : 0 ... 38251 =      0.000 ...   298.836 secs
Ready.
Reading 0 ... 38251  =      0.000 ...   298.836 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 42 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 42.00 Hz
- Upper transition bandwidth: 10.50 Hz (-6 dB cutoff frequency: 47.25 Hz)
- Filter length: 213 samples (1.664 s)

Opening raw data file ThesisFolder\GAMEEMO_FIF\S01G3AllChannels_raw.fif...
    Range : 0 ... 38251 =      0.000 ...   298.836 secs
Ready.
Reading 0 ... 38251  =      0.000 ...   298.836 secs...
Filtering raw data in 1 contiguous segment
Setting 

  3%|██▏                                                                               | 3/112 [00:00<00:15,  7.07it/s]

...

 97%|█████████████████████████████████████████████████████████████████████████████▊  | 109/112 [00:09<00:00, 12.02it/s]

Opening raw data file ThesisFolder\GAMEEMO_FIF\S28G2AllChannels_raw.fif...
    Range : 0 ... 38251 =      0.000 ...   298.836 secs
Ready.
Reading 0 ... 38251  =      0.000 ...   298.836 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 42 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 42.00 Hz
- Upper transition bandwidth: 10.50 Hz (-6 dB cutoff frequency: 47.25 Hz)
- Filter length: 213 samples (1.664 s)

Opening raw data file ThesisFolder\GAMEEMO_FIF\S28G3AllChannels_raw.fif...
    Range : 0 ... 38251 =      0.000 ...   298.836 secs
Ready.
Reading 0 ... 38251  =      0.000 ...   298.836 secs...
Filtering raw data in 1 contiguous segment
Setting 

 99%|███████████████████████████████████████████████████████████████████████████████▎| 111/112 [00:09<00:00, 12.14it/s]

Opening raw data file ThesisFolder\GAMEEMO_FIF\S28G4AllChannels_raw.fif...
    Range : 0 ... 38251 =      0.000 ...   298.836 secs
Ready.
Reading 0 ... 38251  =      0.000 ...   298.836 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 42 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 42.00 Hz
- Upper transition bandwidth: 10.50 Hz (-6 dB cutoff frequency: 47.25 Hz)
- Filter length: 213 samples (1.664 s)



100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [00:09<00:00, 11.64it/s]


In [12]:
print("Files: ", rawArray.shape[0], "\n",
     "Channels: ", rawArray.shape[1], "\n",
     "Samples: ", rawArray.shape[2], "\n",)

Files:  112 
 Channels:  14 
 Samples:  38252 



# Cochleagram

In [15]:
sample_data = rawArray[100][12][:2000]

# cochleagram Params
n = 64
low_lim = 8
high_lim = 42
sample_factor = 1

# generate cochleagram
cochleagram = cgram.cochleagram(sample_data, 128, n, low_lim, high_lim, sample_factor)

# display cochleagram
plt.subplot(211)
plt.imshow(cochleagram, aspect='auto', origin='lower', extent=[0, len(sample_data)/128, low_lim, high_lim])
plt.colorbar()
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Cochleagram')

plt.subplot(212)
plt.plot(sample_data)
plt.xlabel("samples (n)")
plt.ylabel("values")

plt.show()

# Read Data files (fif) and chop into epochs

# Read epoch files

## goofy ahh work